In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi']= 150

In [3]:
import numpy as np
import pandas as pd

import ipywidgets as widgets
from ipywidgets import interact, fixed


# Data
from transat.data import HYPOTHETICAL_SUBMISSION_DATE
from transat.data.load import download_historical, load_historical
from transat.data.split import split_historical
from transat.data.transform import preprocess_historical_basic, dataframe_to_array, NPI_COLS, CASES_COL

# Metric
from transat.metric import mae

# Scenario/Simulation
from transat.data.scenario import generate_scenario

In [4]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]

In [5]:
def download_csv(url, path, prefix):
    """
    Downloads a CSV from 'url', saves it to 'path' folder with filename 'prefix'_DD-MM-YYYY formatted at today's date
    """
    import requests
    import os
    import datetime
    response = requests.get(url, allow_redirects=True)
    today = str(datetime.date.today())
    os.makedirs(path, exist_ok=True)
    filepath = f'{path}/{prefix}_{today}.csv'
    open(filepath, 'wb').write(response.content)
    return filepath

def update_owid(path):
    """
    Updates Our World In Data database and saves it to 'path' folder. Renames it to owid_DD-MM-YYYY with today's date
    """
    filepath = download_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', path, 'owid')
    print(f'Downloaded Our World In Data Coronavirus data to \n\t{filepath}')
    return filepath

# Create a "owid-data"
owid_filepath = update_owid("data")

Downloaded Our World In Data Coronavirus data to 
	data/owid_2020-12-15.csv


In [20]:
df_owid = pd.read_csv(owid_filepath)

countryCode_2_population = {iso:pop for iso,pop in zip(df_owid.iso_code, df_owid.population)}

# df_owid = pd.DataFrame({
#     "CountryCode": df_owid.iso_code.tolist(),
#     "Population": df_owid.population.tolist()
# })

In [25]:
download_historical()
df = load_historical()
df.shape

(92834, 49)

In [26]:
df["Population"] = 1
for ccode, pop in countryCode_2_population.items():
    df.loc[df.CountryCode == ccode, "Population"] = pop
df = df[df.Population > 1]

In [27]:
df.shape

(88995, 50)

In [28]:
def preprocess_historical_with_population(df, cases_col=CASES_COL, npi_cols=NPI_COLS):
    """Create a copy of preprocessed data."""

    df = df.copy()

    # Add RegionID column that combines CountryName and RegionName for easier manipulation of data
    df["GeoID"] = df["CountryName"] + "__" + df["RegionName"].astype(str)

    # Add new cases column
    df["NewCases"] = df.groupby("GeoID").ConfirmedCases.diff().fillna(0)
    
    df["NewCases"] = df.NewCases / df.Population

    # Keep only columns of interest
    id_cols = ["CountryName", "RegionName", "GeoID", "Date"]

    df = df[id_cols + cases_col + npi_cols]

    # Fill any missing case values by interpolation and setting NaNs to 0
    df.update(
        df.groupby("GeoID").NewCases.apply(lambda group: group.interpolate()).fillna(0)
    )

    # Fill any missing NPIs by assuming they are the same as previous day
    for npi_col in npi_cols:
        df.update(df.groupby("GeoID")[npi_col].ffill().fillna(0))

    return df

In [29]:
df = preprocess_historical_with_population(df)
df.shape

(88995, 17)

In [53]:
def plot_new_cases(geo_id, df):
    
    nc = df.loc[df.GeoID == geo_id, "NewCases"].tolist()
    
    plt.figure()
    plt.plot(nc)
    plt.ylim(-0.0001,0.002)
    plt.show()
    
geo_ids = sorted(df.GeoID.unique())

w_geo_id = widgets.Dropdown(
    options=geo_ids,
    value='France__nan',
    description='GeoID:',
    disabled=False
)

interact(plot_new_cases, geo_id=w_geo_id, df=fixed(df))

interactive(children=(Dropdown(description='GeoID:', index=84, options=('Afghanistan__nan', 'Albania__nan', 'A…

<function __main__.plot_new_cases(geo_id, df)>

In [31]:
print("Spliting at : ", HYPOTHETICAL_SUBMISSION_DATE)
df_train, df_test = split_historical(df, HYPOTHETICAL_SUBMISSION_DATE)

Spliting at :  2020-07-31


In [32]:
nb_lookback_days = 30
sequence_format = True
neg_npis = False

(X_train, y_train), (X_cols, y_col) = dataframe_to_array(df_train, nb_lookback_days=nb_lookback_days,
    sequence_format=sequence_format, neg_npis=neg_npis)
(X_test, y_test), _ = dataframe_to_array(df_test, nb_lookback_days=nb_lookback_days,
    sequence_format=sequence_format, neg_npis=neg_npis)

# X_train, y_train = X_train.reshape(X_train.shape[0], -1), y_train.reshape(-1)
# X_test, y_test = X_test.reshape(X_test.shape[0], -1), y_test.reshape(-1)

print("X_train shape: ", np.shape(X_train))
print("y_train shape: ", np.shape(y_train))
print()
print("X_test  shape: ", np.shape(X_test))
print("y_test  shape: ", np.shape(y_test))

X_train shape:  (46410, 30, 13)
y_train shape:  (46410, 1, 1)

X_test  shape:  (26775, 30, 13)
y_test  shape:  (26775, 1, 1)


In [33]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.

from sklearn.model_selection import train_test_split

# from sklearn.utils import shuffle
# X_train, y_train = shuffle(X_train, y_train)

# Split data into train and test sets
X_train, X_valid , y_train, y_valid= train_test_split(
    X_train,
    y_train,
    test_size=0.2,
    random_state=301
)

In [34]:
import tensorflow as tf

class LSTM:
    
    def fit(self, X, y, epochs=1, batch_size=32):
        
        # Build Model
        if not(hasattr(self, "model")):
            input_shape = X.shape[1:]
            self.model = self.build_model(input_shape)
            
        # Pre-process data
        self.fit_preprocess(X, y)
        X, y = self.transform(X, y)
        
        # Fit Model
        self.model.fit(X, y, epochs=epochs, batch_size=batch_size)
        
    def predict(self, X):
        X = self.transform(X)
        
        y = self.model.predict(X).reshape(-1)
        
        # Inverse preprocessing
#         y = y * self.std[0] + self.mean[0]
        y = y * (self.max[0] - self.min[0]) + self.min[0]
        
        return y
    
    def fit_preprocess(self, X, y):
        # MinMax (x - min) / (max - min)
        self.min = X.reshape(-1, X.shape[-1]).min(axis=0)
        self.max = X.reshape(-1, X.shape[-1]).max(axis=0)

        # Normalization
#         self.mean = X.reshape(-1, X.shape[-1]).mean(axis=0)
#         self.std = X.reshape(-1, X.shape[-1]).std(axis=0)

        
    
    
    def transform(self, X, y=None):
        X = (X - self.min) / (self.max - self.min)
#         X = (X - self.mean) / self.std
        if y is not None:
            y = (y - self.min[0]) / (self.max[0] - self.min[0])
#             y = (y - self.mean[0]) / self.std[0]
            return X, y
        else:
            return X
    
    def build_model(self, input_shape):

        input = tf.keras.Input(shape=input_shape, name='input')
        x = tf.keras.layers.LSTM(32, return_sequences=True)(input)
        x = tf.keras.layers.LSTM(32)(x)
#         x = tf.keras.layers.Dense(64, activation='relu')(x)
        x = tf.keras.layers.Dense(64, activation='relu')(x)
        output = tf.keras.layers.Dense(1, activation=None, name='output')(x)
        model = tf.keras.Model(inputs=[input], outputs=[output])

        model.compile(
            loss=tf.losses.MeanSquaredError(),
            optimizer=tf.optimizers.Adam(),
            # metrics=[tf.metrics.MeanAbsoluteError()]
        )

        return model
    
model = LSTM()
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
1161/1161 [==============================] - 22s 19ms/step - loss: 4.5067e-04
Epoch 2/5
1161/1161 [==============================] - 23s 20ms/step - loss: 2.4966e-04
Epoch 3/5
1161/1161 [==============================] - 26s 23ms/step - loss: 2.1602e-04
Epoch 4/5
1161/1161 [==============================] - 25s 21ms/step - loss: 1.7829e-04
Epoch 5/5
1161/1161 [==============================] - 24s 21ms/step - loss: 1.8632e-04


In [12]:
model.fit(X_train, y_train, epochs=30)

Epoch 1/30
1211/1211 [==============================] - 5s 5ms/step - loss: 7.2632e-05
Epoch 2/30
1211/1211 [==============================] - 6s 5ms/step - loss: 7.0572e-05
Epoch 3/30
1211/1211 [==============================] - 5s 5ms/step - loss: 7.2640e-05
Epoch 4/30
1211/1211 [==============================] - 5s 5ms/step - loss: 6.5973e-05
Epoch 5/30
1211/1211 [==============================] - 6s 5ms/step - loss: 6.9094e-05
Epoch 6/30
1211/1211 [==============================] - 6s 5ms/step - loss: 6.5384e-05
Epoch 7/30
1211/1211 [==============================] - 5s 5ms/step - loss: 6.3773e-05
Epoch 8/30
1211/1211 [==============================] - 6s 5ms/step - loss: 5.7737e-05
Epoch 9/30
1211/1211 [==============================] - 6s 5ms/step - loss: 5.5563e-05
Epoch 10/30
1211/1211 [==============================] - 6s 5ms/step - loss: 5.5472e-05
Epoch 11/30
1211/1211 [==============================] - 6s 5ms/step - loss: 5.8449e-05
Epoch 12/30
1211/1211 [==================

In [13]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

valid_preds = model.predict(X_valid)
valid_preds = np.maximum(valid_preds, 0) # Don't predict negative cases
print('Valid MAE:', mae(valid_preds, y_valid))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 939.424174252276
Valid MAE: 901.3265630273827
Test MAE: 3553.7698564215925


In [36]:
def simulate_scenario(model, X_scenario, y_scenario, seq=False):
    # Simulate scenario

    X_sim = X_scenario.copy()
    X_sim_cases = X_sim[:,:,:1]
    X_sim_npis = X_sim[:,:,1:]
    y_sim = np.zeros(np.shape(y_scenario))

    nb_lookback_days = X_sim.shape[1]

    for d in range(y_sim.shape[1]):
        
        if seq:
            y = model.predict(X_sim)
        else:
            y = model.predict(X_sim.reshape(1,-1))
        y_sim[0,d,0] = max(y[0], 0)

        # Assuming constant NPIs here
        X_sim_npis = np.concatenate([X_sim_npis[:,1:], X_sim_npis[:,-1:]], axis=1)
        X_sim_cases = np.concatenate([X_sim_cases[:,1:], y.reshape(-1, 1, 1)], axis=1)

        X_sim =  np.concatenate([X_sim_cases, X_sim_npis], axis=-1)
        X_sim = np.array(X_sim)
    
    return y_sim

In [37]:
def viz_scenario(geo_id, X_scenario, y_scenario, y_sim):
    mae_error = mae(y_scenario, y_sim)

    plt.figure()
    plt.title(geo_id)

    plot_input_x = np.arange(X_scenario.shape[1])
    plot_input_y = X_scenario[:,:,:1].reshape(-1)

    plt.plot(plot_input_x, plot_input_y, label="Input Scenario")

    plot_output_x = np.arange(y_scenario.shape[1])+X_scenario.shape[1]
    plot_output_x = np.concatenate([plot_input_x[-1:], plot_output_x])
    plot_output_y = np.concatenate([plot_input_y[-1:], y_scenario.reshape(-1)])
    plt.plot(plot_output_x, plot_output_y, label="Output Scenario")


    plot_output_y = np.concatenate([plot_input_y[-1:], y_sim.reshape(-1)])
    plt.plot(plot_output_x, plot_output_y, label="Output Simulation")

    plt.ylabel("New Cases")
    plt.xlabel("Days")
    ax = plt.gca()
    plt.text(0.3, 0.5, f"$MAE={mae_error:.2f}$", transform=ax.transAxes)
    plt.legend()
    plt.show()

In [38]:
def interactive_scenario(geo_id, model, seq=True):
    nb_future_days=30

    X_scenario , y_scenario = generate_scenario(df_train, df_test, geo_id, nb_lookback_days=nb_lookback_days,
        nb_future_days=nb_future_days, sequence_format=sequence_format)

    y_sim = simulate_scenario(model, X_scenario, y_scenario, seq=seq)

    viz_scenario(geo_id, X_scenario, y_scenario, y_sim)
    
geo_ids = sorted(df.GeoID.unique())

w_geo_id = widgets.Dropdown(
    options=geo_ids,
    value='France__nan',
    description='GeoID:',
    disabled=False
)

interact(interactive_scenario, geo_id=w_geo_id, model=fixed(model), seq=fixed(True))

interactive(children=(Dropdown(description='GeoID:', index=84, options=('Afghanistan__nan', 'Albania__nan', 'A…

<function __main__.interactive_scenario(geo_id, model, seq=True)>

In [39]:
countryCode_2_population["FRA"]

65273512.0